In [1]:
import numpy as np
import pytest
from numpy.typing import NDArray
from ska_sdp_datamodels.visibility import Visibility as RASCILVisibility
from typing_extensions import assert_never

from karabo.imaging.imager_base import DirtyImagerConfig
from karabo.imaging.imager_oskar import OskarDirtyImager
from karabo.imaging.imager_rascil import RascilDirtyImager
from karabo.imaging.util import (
    auto_choose_dirty_imager_from_sim,
    auto_choose_dirty_imager_from_vis,
)
from karabo.simulation.interferometer import InterferometerSimulation
from karabo.simulation.observation import Observation
from karabo.simulation.sky_model import SkyModel
from karabo.simulation.telescope import Telescope
from karabo.simulation.visibility import Visibility
from karabo.imaging.imager_oskar import OskarDirtyImager, OskarDirtyImagerConfig
from karabo.simulator_backend import SimulatorBackend
from astropy.io import fits
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.use('Tkagg')

The RASCIL data directory is not available - continuing but any simulations will fail


In [13]:
#telescope_path = '/home/rohit/skao_repo/ska1mid.tm'
#telescope_path = '/home/rohit/skao_repo/ska1low.tm'
#telescope_path = '/home/rohit/skao_repo/ska-low-AA0.5.tm'
#telescope_path = '/home/rohit/skao_repo/ska-mid-AA0.5.tm'
#telescope_path = '/home/rohit/skao_repo/ska-low-AA1.tm'
#telescope_path = '/home/rohit/skao_repo/ska-mid-AA1.tm'
#telescope_path = '/home/rohit/skao_repo/ska-low-AA2.tm'
#telescope_path = '/home/rohit/skao_repo/ska-mid-AA2.tm'
telescope_path = '/home/rohit/skao_repo/ska-low-AAstar.tm'
#telescope_path = '/home/rohit/skao_repo/ska-mid-AAstar.tm'
telescope=Telescope.read_OSKAR_tm_file(telescope_path)
telescope.read_OSKAR_tm_file(telescope_path)
telescope.plot_telescope()
plt.show()

In [102]:
#telescope_path = '/media/rohit/sdata/ska-solar-files/meerkat.tm'
telescope_path = '/media/rohit/sdata/ska-solar-files/ska1mid.tm'
#telescope_path = '/media/rohit/sdata/ska-solar-files/mwa.phase1.tm'
#telescope_path = '/media/rohit/sdata/ska-solar-files/ska1low.tm'
telescope_name='skamid_snap0_500MHz'
hour_=0
minutes_=1
noise_enable_ = False
enable_array_beam=False
start_frequency_hz_= 500.e6
point_source_bool = False # tag 1 / tag 0 is ideal case
random_points = False # tag 2
if(random_points==True):
    size_ = 100
else:
    size_ = 1
point_flux = 100.e4

In [103]:
path = '/media/rohit/sdata/ska-solar-files/sim/'
aa=fits.open(path+'20151203_240MHz_psimas.fits')
solar_map=aa[0].data;solar_map_jy=solar_map/np.nanmax(solar_map)*20*1.e4*(start_frequency_hz_/2.4e8)**2
ra_sun_center=249.141666667;dec_sun_center=21.986 #16 34 34.52 -21 59 09.7
ra_grid,dec_grid=np.meshgrid((np.arange(256)-128)*22.5/3600.,(np.arange(256)-128)*22.5/3600.)
ra_grid=ra_grid+ra_sun_center;dec_grid=dec_grid+dec_sun_center
idx=np.where(solar_map>0.001*np.nanmax(solar_map))
sky_model_ra=ra_grid[idx];sky_model_dec=dec_grid[idx];flux=solar_map_jy[idx]
print(flux.max())

868055.5555555557


In [104]:
layout=np.loadtxt(telescope_path+'/layout.txt')
nant=len(layout)
nb=int(nant*(nant-1)*0.5)
print(nb)
base_length=[0]*nb
k=0
for i in range(nant):
    for j in range(i,nant):
        if(i!=j):
            base_length[k] = np.sqrt((layout[i][0]-layout[j][0])**2 + (layout[i][1]-layout[j][1])**2)
            k=k+1
base_length=np.array(base_length)
beam_size_arcsec = 3.e8/start_frequency_hz_/base_length.max()*180/np.pi*3600
print(base_length.max(), beam_size_arcsec)

130816
65412.89611017922 1.8919645988430576


In [105]:
# Simulation starts 
sky = SkyModel()
sky_data = np.array([sky_model_ra, sky_model_dec, flux,np.zeros(len(flux)), \
    np.zeros(len(flux)),np.zeros(len(flux)), np.zeros(len(flux)),np.zeros(len(flux)), \
np.zeros(len(flux)),np.zeros(len(flux)), np.zeros(len(flux)),np.zeros(len(flux))]).T
sky_data=sky_data[0:16000,:]
# Add Point Soure
add_source = np.zeros((size_,12))
if(point_source_bool):
    add_source[0][0] = ra_sun_center # RA
    add_source[0][1] = dec_sun_center # DEC
    add_source[0][2] = point_flux # Flux
    sky_data1 = np.vstack((sky_data,add_source))
elif(random_points):
    ra_point_array = np.random.uniform(low=ra_sun_center-0.3, high=ra_sun_center+0.3, size=size_)
    dec_point_array = np.random.uniform(low=dec_sun_center-0.3, high=dec_sun_center+0.3, size=size_)
    flux_array = np.random.uniform(low=1.e1, high=1.e2, size=size_)
    add_source[:,0] = ra_point_array # RA
    add_source[:,1] = dec_point_array # DEC
    add_source[:,2] = flux_array # Flux  
    sky_data1 = np.vstack((sky_data,add_source))
else:
    sky_data1 = sky_data
sky.add_point_sources(sky_data1)
np.savetxt(path+"source_"+telescope_name+".txt", add_source.T)
#telescope_name="SKA1MID"
#telescope=Telescope.get_SKA1_LOW_Telescope()
dtime=datetime(2000, 1, 1, 10, 0, 00, 0) # MeerKAT/ SKA-mid
#dtime=datetime(2000, 1, 1, 4, 0, 00, 0) # MWA / SKA-low
backend=SimulatorBackend.OSKAR
#telescope = Telescope.constructor(telescope_name, backend=backend)
telescope=Telescope.read_OSKAR_tm_file(telescope_path)
telescope.read_OSKAR_tm_file(telescope_path)
telescope.plot_telescope()
plt.show()

In [106]:

simulation = InterferometerSimulation(vis_path=path+'solar_'+telescope_name+'.vis', 
    channel_bandwidth_hz=2e7, time_average_sec=1, noise_enable=noise_enable_, use_gpus=True,
    noise_seed="time", noise_freq="Range", noise_rms="Range", 
    noise_start_freq=1.e9, noise_inc_freq=1.e8, noise_number_freq=24, 
    noise_rms_start=0, noise_rms_end=0, enable_numerical_beam=enable_array_beam,
    enable_array_beam=enable_array_beam) 
observation = Observation(mode='Tracking',phase_centre_ra_deg=ra_sun_center, start_date_and_time=dtime, 
    length=timedelta(hours=hour_, minutes=minutes_, seconds=0, milliseconds=0), 
    phase_centre_dec_deg=dec_sun_center, number_of_time_steps=100, 
    start_frequency_hz=start_frequency_hz_, frequency_increment_hz=1e6, 
    number_of_channels=1, ) 
visibility = simulation.run_simulation(telescope, sky, observation, backend=backend)
visibility.write_to_file(path+"solar_"+telescope_name+".ms") 

# IMAGING 
cellsize_rad=3.878509448876288e-05 # in rad
cellsize_arcsec=cellsize_rad*180/np.pi*3600
print(cellsize_arcsec)
dirty_imager = OskarDirtyImager(
    OskarDirtyImagerConfig(
        imaging_npixel=2048,
        imaging_cellsize=cellsize_rad,
    ))
dirty_image = dirty_imager.create_dirty_image(visibility,output_fits_path=path+"solar_"+telescope_name+".fits") 
npix_per_beam = beam_size_arcsec/cellsize_arcsec
print(npix_per_beam,beam_size_arcsec,cellsize_arcsec,base_length.max())

Creating /tmp/karabo-STM-rohit-TRnaSBphUX/interferometer-9d47Z5nOTT for interferometer disk-cache.


KeyboardInterrupt: 

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

d,h=fits.getdata(path+"solar_"+telescope_name+".fits",header=True)
l1 = h['CRVAL1'] + h['CDELT1']*1024
l2 = h['CRVAL1'] - h['CDELT1']*1024
r1 = h['CRVAL2'] + h['CDELT2']*1024
r2 = h['CRVAL2'] - h['CDELT2']*1024
d=d[:,:,::-1,::-1]

f,((ax0,ax1),(ax2,ax3))=plt.subplots(2,2)
im0=ax0.imshow(solar_map_jy/1.e4,origin='lower', extent=[ra_grid[0][0],ra_grid[0][-1],dec_grid[:,0][0],dec_grid[:,0][-1]])
divider = make_axes_locatable(ax0)
cax = divider.append_axes('right', size='5%', pad=0.05)
f.colorbar(im0, cax=cax, orientation='vertical',label = 'SFU')
im1=ax1.imshow(d[0][0]/(np.pi*npix_per_beam**2)/1.e4,origin='lower', extent = [l1,l2,r1,r2])
ax1.set_xlim(ra_grid[0][0],ra_grid[0][-1])
ax1.set_ylim(dec_grid[:,0][0],dec_grid[:,0][-1])
divider = make_axes_locatable(ax1)
cax = divider.append_axes('right', size='5%', pad=0.05)
f.colorbar(im1, cax=cax, orientation='vertical',label = 'SFU')
#ax2.plot(ra_point_array,dec_point_array,'o',color='k')
c1 = plt.Circle( ( ra_sun_center, dec_sun_center ), 0.25, color='red',fill = None )
ax2.add_patch(c1)
ax3.imshow(solar_map_jy,origin='lower', extent=[ra_grid[0][0],ra_grid[0][-1],dec_grid[:,0][0],dec_grid[:,0][-1]])
#ax3.plot(ra_point_array,dec_point_array,'o',color='k')
ax0.set_xlabel('R.A. (Deg)');ax1.set_xlabel('R.A. (Deg)');ax2.set_xlabel('R.A. (Deg)');ax3.set_xlabel('R.A. (Deg)')
ax0.set_ylabel('DEC (deg)');ax1.set_ylabel('DEC (deg)');ax2.set_ylabel('DEC (deg)');ax3.set_ylabel('DEC (deg)')
plt.show()

In [ ]:
#telescope.plot_telescope()
#plt.show()

In [ ]:
print(npix_per_beam)

0.0982856464058907
